In [1]:
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '..')

# for saving
import os

from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

from src import setup
# setup.use_gpu()

In [2]:
from src import ( 
    measurements as measure,
    density_matrix as DM,
    simulation as sim,
    orders,
    order_rules,
    random_unitary,
    simulation)

from Scripts import simulation_CLI as cleo

In [3]:
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors 

import matplotlib.animation as animation
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '../..')
import h5py
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from src import ket as ket
from src import density_matrix as DM

In [4]:
#pick={therm,inhomov1,inhomov2,pure} pick one of these to properly store images with the correct labels
dataset_label = "pure"

In [5]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/greedy")

In [ ]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/random")

In [7]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/landscape_maximizes")

In [8]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/mimic")

In [ ]:
newdata1 = dict(h5py.File("../data/random/random.hdf5"))
#newdata2 = dict(h5py.File("../data/greedy/greedy.hdf5"))
#newdata3 = dict(h5py.File("../data/mimic/mimic.hdf5"))
#newdata4 = dict(h5py.File("../data/landscape_maximizes/landscape_maximizes.hdf5"))

In [9]:
def get_pops(data, n_qubits, connectivity,update_rule):
    #dimension 0 is each trial
    #dimension 1 is each time step
    #dimension 2 is each qubit
    result = []
    for trial in data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1']:
        seed = trial.split(' ')[-1]
        dat = dict(data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1'][f'unitary seed {seed}'][f'ordering seed {update_rule}']['pops'])
        dat = {int(k.split('(')[0]): dat[k] for k in dat}
        dat = np.array([np.array([dat[k][subkey][()] for subkey in sorted(dat[k])]) for k in sorted(dat)])
        result.append(dat)
    return(np.array(result))

def get_2_qbit_dms(data, n_qubits, connectivity,update_rule):
    basis = ket.canonical_basis(2)
    #dimension 0 is each trial
    #dimension 1 is each time step (recall that sampling step is 5 by default)
    #dimension 2 is qubit pair
    #dimension 3 and 4 is the 2 qubit density matrix
    result = []
    
    def to_tuple(string):
        tuple_elements = string.strip('()').split(',')
        return tuple(int(elem.strip()) for elem in tuple_elements)

    for trial in data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1']:
        seed = trial.split(' ')[-1]
        dat = dict(data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1'][f'unitary seed {seed}'][f'ordering seed {update_rule}']['two_qubit_dms'])
        dat = {int(k.split('(')[0]): dat[k] for k in dat}
        
        dat = np.array([{to_tuple(subkey):DM.DensityMatrix(dat[k][subkey],basis) for subkey in sorted(dat[k])} for k in sorted(dat)])
        result.append(dat)
    return(np.array(result))

In [ ]:
#Finds the pops data for each rule, all the trials
num=8 #insert num of qubits

In [ ]:
pops_random_c2 = get_pops(newdata1, num, "c2_2local", "random")
pops_random_c4 = get_pops(newdata1, num, "c4_2local", "random")
pops_random_c5 = get_pops(newdata1, num, "c5_2local", "random")
pops_random_c6 = get_pops(newdata1, num, "c6_2local", "random")
pops_random_cn = get_pops(newdata1, num, "cN_2local", "random")

In [ ]:
pops_greedy_c2 = get_pops(newdata2, num, "c2_2local", "greedy")
pops_greedy_c4 = get_pops(newdata2, num, "c4_2local", "greedy")
pops_greedy_c5 = get_pops(newdata2, num, "c5_2local", "greedy")
pops_greedy_c6 = get_pops(newdata2, num, "c6_2local", "greedy")
pops_greedy_cn = get_pops(newdata2, num, "cN_2local", "greedy")

In [ ]:
pops_mimic_c2 = get_pops(newdata3, num, "c2_2local", "mimic")
pops_mimic_c4 = get_pops(newdata3, num, "c4_2local", "mimic")
pops_mimic_c5 = get_pops(newdata3, num, "c5_2local", "mimic")
pops_mimic_c6 = get_pops(newdata3, num, "c6_2local", "mimic")
pops_mimic_cn = get_pops(newdata3, num, "cN_2local", "mimic")

In [ ]:
pops_landscape_maximizes_c2 = get_pops(newdata4, num, "c2_2local", "landscape_maximizes")
pops_landscape_maximizes_c4 = get_pops(newdata4, num, "c4_2local", "landscape_maximizes")
pops_landscape_maximizes_c5 = get_pops(newdata4, num, "c5_2local", "landscape_maximizes")
pops_landscape_maximizes_c6 = get_pops(newdata4, num, "c6_2local", "landscape_maximizes")
pops_landscape_maximizes_cn = get_pops(newdata4, num, "cN_2local", "landscape_maximizes")

In [ ]:
#Ensemble average across 100 data sets

In [ ]:
def ensemble_averaged_one_point_measures_mean_std(datasets):
    # Assuming datasets is a list of 100 datasets, where each dataset is a 2D array of shape (100, N)
    # Stack datasets into a 3D array
    stacked_data = np.stack(datasets)  # Shape (100, 100, N)

    # Take the ensemble average along the first axis (averaging over all datasets)
    ensemble_average = np.mean(stacked_data, axis=0)  # Shape (100, N)

    # Calculate the standard deviation along the first axis
    ensemble_std = np.std(stacked_data, axis=0) 
    return (ensemble_average, ensemble_std)

In [ ]:
pops_random_c2_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_c2)[0]
pops_random_c4_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_c4)[0]
pops_random_c5_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_c5)[0]
pops_random_c6_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_c6)[0]
pops_random_cn_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_cn)[0]

In [ ]:
pops_greedy_c2_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_c2)[0]
pops_greedy_c4_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_c4)[0]
pops_greedy_c5_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_c5)[0]
pops_greedy_c6_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_c6)[0]
pops_greedy_cn_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_cn)[0]

In [ ]:
pops_mimic_c2_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_c2)[0]
pops_mimic_c4_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_c4)[0]
pops_mimic_c5_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_c5)[0]
pops_mimic_c6_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_c6)[0]
pops_mimic_cn_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_cn)[0]

In [ ]:
pops_landscape_maximizes_c2_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_c2)[0]
pops_landscape_maximizes_c4_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_c4)[0]
pops_landscape_maximizes_c5_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_c5)[0]
pops_landscape_maximizes_c6_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_c6)[0]
pops_landscape_maximizes_cn_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_cn)[0]

In [ ]:
def extractable_work_of_one_trial(pops,trial_index):
    return ext_work= np.array([[measure.extractable_work_of_each_qubit_from_pops(p) for p in time_slice ]for time_slice in pops[trial]])

In [ ]:
def extractable_work_of_all_trials(pops):
    ext_work_all = []
    for trial in range(100):
        ext_work_all.append(extractable_work_of_one_trial(pops,trial_index))
    return ext_work_all

In [ ]:
#Ext work for all trials of a particular rule and connectivity

In [ ]:
ext_work_all_random_c2 = extractable_work_of_all_trials(pops_random_c2)
ext_work_all_random_c4 = extractable_work_of_all_trials(pops_random_c4)
ext_work_all_random_c5 = extractable_work_of_all_trials(pops_random_c5)
ext_work_all_random_c6 = extractable_work_of_all_trials(pops_random_c6)
ext_work_all_random_cn = extractable_work_of_all_trials(pops_random_cn)

In [ ]:
ext_work_all_greedy_c2 = extractable_work_of_all_trials(pops_greedy_c2)
ext_work_all_greedy_c4 = extractable_work_of_all_trials(pops_greedy_c4)
ext_work_all_greedy_c5 = extractable_work_of_all_trials(pops_greedy_c5)
ext_work_all_greedy_c6 = extractable_work_of_all_trials(pops_greedy_c6)
ext_work_all_greedy_cn = extractable_work_of_all_trials(pops_greedy_cn)

In [ ]:
ext_work_all_mimic_c2 = extractable_work_of_all_trials(pops_mimic_c2)
ext_work_all_mimic_c4 = extractable_work_of_all_trials(pops_mimic_c4)
ext_work_all_mimic_c5 = extractable_work_of_all_trials(pops_mimic_c5)
ext_work_all_mimic_c6 = extractable_work_of_all_trials(pops_mimic_c6)
ext_work_all_mimic_cn = extractable_work_of_all_trials(pops_mimic_cn)

In [ ]:
ext_work_all_landscape_maximizes_c2 = extractable_work_of_all_trials(pops_landscape_maximizes_c2)
ext_work_all_landscape_maximizes_c4 = extractable_work_of_all_trials(pops_landscape_maximizes_c4)
ext_work_all_landscape_maximizes_c5 = extractable_work_of_all_trials(pops_landscape_maximizes_c5)
ext_work_all_landscape_maximizes_c6 = extractable_work_of_all_trials(pops_landscape_maximizes_c6)
ext_work_all_landscape_maximizes_cn = extractable_work_of_all_trials(pops_landscape_maximizes_cn)

In [ ]:
#Extractable work of the ensembled average trial per connectivity per rule W(<p>) where p is the landscape

In [ ]:
ext_work_of_ens_avg_random_c2 = extractable_work_of_one_trial(pops_random_c2_ens_avg,0)
ext_work_of_ens_avg_random_c4 = extractable_work_of_one_trial(pops_random_c4_ens_avg,0)
ext_work_of_ens_avg_random_c5 = extractable_work_of_one_trial(pops_random_c5_ens_avg,0)
ext_work_of_ens_avg_random_c6 = extractable_work_of_one_trial(pops_random_c6_ens_avg,0)
ext_work_of_ens_avg_random_cn = extractable_work_of_one_trial(pops_random_cn_ens_avg,0)

In [ ]:
ext_work_of_ens_avg_greedy_c2 = extractable_work_of_one_trial(pops_random_c2_ens_avg,0)
ext_work_of_ens_avg_greedy_c2 = extractable_work_of_one_trial(pops_random_c4_ens_avg,0)
ext_work_of_ens_avg_greedy_c2 = extractable_work_of_one_trial(pops_random_c5_ens_avg,0)
ext_work_of_ens_avg_greedy_c2 = extractable_work_of_one_trial(pops_random_c6_ens_avg,0)
ext_work_of_ens_avg_greedy_c2 = extractable_work_of_one_trial(pops_random_cn_ens_avg,0)

In [ ]:
ext_work_of_ens_avg_mimic_c2 = extractable_work_of_one_trial(pops_mimic_c2_ens_avg,0)
ext_work_of_ens_avg_mimic_c4 = extractable_work_of_one_trial(pops_mimic_c4_ens_avg,0)
ext_work_of_ens_avg_mimic_c5 = extractable_work_of_one_trial(pops_mimic_c5_ens_avg,0)
ext_work_of_ens_avg_mimic_c6 = extractable_work_of_one_trial(pops_mimic_c6_ens_avg,0)
ext_work_of_ens_avg_mimic_cn = extractable_work_of_one_trial(pops_mimic_cn_ens_avg,0)

In [ ]:
ext_work_of_ens_avg_landscape_maximizes_c2 = extractable_work_of_one_trial(pops_landscape_maximizes_c2_ens_avg,0)
ext_work_of_ens_avg_landscape_maximizes_c4 = extractable_work_of_one_trial(pops_landscape_maximizes_c4_ens_avg,0)
ext_work_of_ens_avg_landscape_maximizes_c5 = extractable_work_of_one_trial(pops_landscape_maximizes_c5_ens_avg,0)
ext_work_of_ens_avg_landscape_maximizes_c6 = extractable_work_of_one_trial(pops_landscape_maximizes_c6_ens_avg,0)
ext_work_of_ens_avg_landscape_maximizes_cn = extractable_work_of_one_trial(pops_landscape_maximizes_cn_ens_avg,0)

In [ ]:
#Ensemble average of extractable work computed for each trial per connectivity per rule <W(p)> where p is the landscape

In [ ]:
ens_avg_ext_work_random_c2 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_random_c2)[0]
ens_avg_ext_work_random_c4 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_random_c4)[0]
ens_avg_ext_work_random_c5 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_random_c5)[0]
ens_avg_ext_work_random_c6 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_random_c6)[0]
ens_avg_ext_work_random_cn = ensemble_averaged_one_point_measures_mean_std(ext_work_all_random_cn)[0]

In [ ]:
ens_avg_ext_work_greedy_c2 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_greedy_c2)[0]
ens_avg_ext_work_greedy_c4 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_greedy_c4)[0]
ens_avg_ext_work_greedy_c5 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_greedy_c5)[0]
ens_avg_ext_work_greedy_c6 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_greedy_c6)[0]
ens_avg_ext_work_greedy_cn = ensemble_averaged_one_point_measures_mean_std(ext_work_all_greedy_cn)[0]

In [ ]:
ens_avg_ext_work_mimic_c2 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_mimic_c2)[0]
ens_avg_ext_work_mimic_c4 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_mimic_c4)[0]
ens_avg_ext_work_mimic_c5 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_mimic_c5)[0]
ens_avg_ext_work_mimic_c6 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_mimic_c6)[0]
ens_avg_ext_work_mimic_cn = ensemble_averaged_one_point_measures_mean_std(ext_work_all_mimic_cn)[0]

In [ ]:
ens_avg_ext_work_landscape_maximizes_c2 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_landscape_maximizes_c2)[0]
ens_avg_ext_work_landscape_maximizes_c4 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_landscape_maximizes_c4)[0]
ens_avg_ext_work_landscape_maximizes_c5 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_landscape_maximizes_c5)[0]
ens_avg_ext_work_landscape_maximizes_c6 = ensemble_averaged_one_point_measures_mean_std(ext_work_all_landscape_maximizes_c6)[0]
ens_avg_ext_work_landscape_maximizes_cn = ensemble_averaged_one_point_measures_mean_std(ext_work_all_landscape_maximizes_cn)[0]

In [ ]:
#Plot ext work per qubit as a function of time 

In [ ]:
#Ensembled average ext work <W(p)>

In [ ]:
#random
# Define a list of colors (you can choose any color names or hex codes)
#for 8Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2']
#for 10 Q
colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF']
#for 12 Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C']
#for 14Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C', '#C4BEB5', '#8E7BBA']
dataset_names = ['c2', 'c4', 'c5', 'c6', 'cn']  # Names corresponding to each dataset
datas=[ens_avg_ext_work_random_c2,ens_avg_ext_work_random_c4,ens_avg_ext_work_random_c5,ens_avg_ext_work_random_c6,ens_avg_ext_work_random_cn]


for data,conn in zip(datas, dataset_names):
    num_time_steps = len(data)-2  # The number of rows in the data corresponds to the number of time steps
    t = np.arange(num_time_steps)  # Create a list of time steps [0, 1, 2, ..., num_time_steps-1]
    
    data=np.array(data[1:499])
    data_transposed = data.T  # Now rows are [x1_all_times, x2_all_times, ..., xN_all_times]
    
    plt.figure(figsize=(12, 8))
    #change this when analyzing pure dataset
    #plt.axhline(y=ytherm10, color='r', linestyle='--', label=f'y = {ytherm10}')

    # Plot each number as a function of t with specified colors
    for i, y_values in enumerate(data_transposed):
        color = colors[i % len(colors)]  # Cycle through the color list
        plt.plot(t, y_values, label=f'$Q_{i+1}$', color=color)
    
    # Add labels and legend
    plt.xlabel('Time (t)',fontsize=14)
    plt.ylabel('Value',fontsize=14)
    plt.title('$<\Delta W(\mathcal{Q}_i) >$ as a function of time',fontsize=16)
    plt.legend(fontsize=12)
    
    # Show the plot
    plt.savefig(f'Ensemble averaged extractable work of network {num}q_conn_{conn}_rule_R1_{dataset_label}.pdf', format='pdf', bbox_inches='tight')

    plt.show()

In [ ]:
#greedy
# Define a list of colors (you can choose any color names or hex codes)
#for 8Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2']
#for 10 Q
colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF']
#for 12 Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C']
#for 14Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C', '#C4BEB5', '#8E7BBA']
dataset_names = ['c2', 'c4', 'c5', 'c6', 'cn']  # Names corresponding to each dataset
datas=[ens_avg_ext_work_greedy_c2,ens_avg_ext_work_greedy_c4,ens_avg_ext_work_greedy_c5,ens_avg_ext_work_greedy_c6,ens_avg_ext_work_greedy_cn]


for data,conn in zip(datas, dataset_names):
    num_time_steps = len(data)-2  # The number of rows in the data corresponds to the number of time steps
    t = np.arange(num_time_steps)  # Create a list of time steps [0, 1, 2, ..., num_time_steps-1]
    
    data=np.array(data[1:499])
    data_transposed = data.T  # Now rows are [x1_all_times, x2_all_times, ..., xN_all_times]
    
    plt.figure(figsize=(12, 8))
    #change this when analyzing pure dataset
    #plt.axhline(y=ytherm10, color='r', linestyle='--', label=f'y = {ytherm10}')

    # Plot each number as a function of t with specified colors
    for i, y_values in enumerate(data_transposed):
        color = colors[i % len(colors)]  # Cycle through the color list
        plt.plot(t, y_values, label=f'$Q_{i+1}$', color=color)
    
    # Add labels and legend
    plt.xlabel('Time (t)',fontsize=14)
    plt.ylabel('Value',fontsize=14)
    plt.title('$<\Delta W(\mathcal{Q}_i) >$ as a function of time',fontsize=16)
    plt.legend(fontsize=12)
    
    # Show the plot
    plt.savefig(f'Ensemble averaged extractable work of network {num}q_conn_{conn}_rule_R2_{dataset_label}.pdf', format='pdf', bbox_inches='tight')

    plt.show()

In [ ]:
#mimic
# Define a list of colors (you can choose any color names or hex codes)
#for 8Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2']
#for 10 Q
colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF']
#for 12 Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C']
#for 14Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C', '#C4BEB5', '#8E7BBA']
dataset_names = ['c2', 'c4', 'c5', 'c6', 'cn']  # Names corresponding to each dataset
datas=[ens_avg_ext_work_mimic_c2,ens_avg_ext_work_mimic_c4,ens_avg_ext_work_mimic_c5,ens_avg_ext_work_mimic_c6,ens_avg_ext_work_mimic_cn]


for data,conn in zip(datas, dataset_names):
    num_time_steps = len(data)-2  # The number of rows in the data corresponds to the number of time steps
    t = np.arange(num_time_steps)  # Create a list of time steps [0, 1, 2, ..., num_time_steps-1]
    
    data=np.array(data[1:499])
    data_transposed = data.T  # Now rows are [x1_all_times, x2_all_times, ..., xN_all_times]
    
    plt.figure(figsize=(12, 8))
    #change this when analyzing pure dataset
    #plt.axhline(y=ytherm10, color='r', linestyle='--', label=f'y = {ytherm10}')

    # Plot each number as a function of t with specified colors
    for i, y_values in enumerate(data_transposed):
        color = colors[i % len(colors)]  # Cycle through the color list
        plt.plot(t, y_values, label=f'$Q_{i+1}$', color=color)
    
    # Add labels and legend
    plt.xlabel('Time (t)',fontsize=14)
    plt.ylabel('Value',fontsize=14)
    plt.title('$<\Delta W(\mathcal{Q}_i) >$ as a function of time',fontsize=16)
    plt.legend(fontsize=12)
    
    # Show the plot
    plt.savefig(f'Ensemble averaged extractable work of network {num}q_conn_{conn}_rule_R3_{dataset_label}.pdf', format='pdf', bbox_inches='tight')

    plt.show()

In [ ]:
#landmax
# Define a list of colors (you can choose any color names or hex codes)
#for 8Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2']
#for 10 Q
colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF']
#for 12 Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C']
#for 14Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C', '#C4BEB5', '#8E7BBA']
dataset_names = ['c2', 'c4', 'c5', 'c6', 'cn']  # Names corresponding to each dataset
datas=[ens_avg_ext_work_landscape_maximizes_c2,ens_avg_ext_work_landscape_maximizes_c4,ens_avg_ext_work_landscape_maximizes_c5,ens_avg_ext_work_landscape_maximizes_c6,ens_avg_ext_work_landscape_maximizes_cn]


for data,conn in zip(datas, dataset_names):
    num_time_steps = len(data)-2  # The number of rows in the data corresponds to the number of time steps
    t = np.arange(num_time_steps)  # Create a list of time steps [0, 1, 2, ..., num_time_steps-1]
    
    data=np.array(data[1:499])
    data_transposed = data.T  # Now rows are [x1_all_times, x2_all_times, ..., xN_all_times]
    
    plt.figure(figsize=(12, 8))
    #change this when analyzing pure dataset
    #plt.axhline(y=ytherm10, color='r', linestyle='--', label=f'y = {ytherm10}')

    # Plot each number as a function of t with specified colors
    for i, y_values in enumerate(data_transposed):
        color = colors[i % len(colors)]  # Cycle through the color list
        plt.plot(t, y_values, label=f'$Q_{i+1}$', color=color)
    
    # Add labels and legend
    plt.xlabel('Time (t)',fontsize=14)
    plt.ylabel('Value',fontsize=14)
    plt.title('$<\Delta W(\mathcal{Q}_i) >$ as a function of time',fontsize=16)
    plt.legend(fontsize=12)
    
    # Show the plot
    plt.savefig(f'Ensemble averaged extractable work of network {num}q_conn_{conn}_rule_R4_{dataset_label}.pdf', format='pdf', bbox_inches='tight')

    plt.show()

In [ ]:
#Ext Work of ensemble average landscape pops

In [ ]:
#random
# Define a list of colors (you can choose any color names or hex codes)
#for 8Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2']
#for 10 Q
colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF']
#for 12 Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C']
#for 14Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C', '#C4BEB5', '#8E7BBA']
dataset_names = ['c2', 'c4', 'c5', 'c6', 'cn']  # Names corresponding to each dataset
datas=[ext_work_of_ens_avg_random_c2,ext_work_of_ens_avg_random_c4,ext_work_of_ens_avg_random_c5,ext_work_of_ens_avg_random_c6,ext_work_of_ens_avg_random_cn]


for data,conn in zip(datas, dataset_names):
    num_time_steps = len(data)-2  # The number of rows in the data corresponds to the number of time steps
    t = np.arange(num_time_steps)  # Create a list of time steps [0, 1, 2, ..., num_time_steps-1]
    
    data=np.array(data[1:499])
    data_transposed = data.T  # Now rows are [x1_all_times, x2_all_times, ..., xN_all_times]
    
    plt.figure(figsize=(12, 8))
    #change this when analyzing pure dataset
    #plt.axhline(y=ytherm10, color='r', linestyle='--', label=f'y = {ytherm10}')

    # Plot each number as a function of t with specified colors
    for i, y_values in enumerate(data_transposed):
        color = colors[i % len(colors)]  # Cycle through the color list
        plt.plot(t, y_values, label=f'$Q_{i+1}$', color=color)
    
    # Add labels and legend
    plt.xlabel('Time (t)',fontsize=14)
    plt.ylabel('Value',fontsize=14)
    plt.title('$\Delta W(<\mathcal{Q}_i >)$ as a function of time',fontsize=16)
    plt.legend(fontsize=12)
    
    # Show the plot
    plt.savefig(f'Extractable work of ensemble averaged network {num}q_conn_{conn}_rule_R1_{dataset_label}.pdf', format='pdf', bbox_inches='tight')

    plt.show()

In [ ]:
#greedy
# Define a list of colors (you can choose any color names or hex codes)
#for 8Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2']
#for 10 Q
colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF']
#for 12 Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C']
#for 14Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C', '#C4BEB5', '#8E7BBA']
dataset_names = ['c2', 'c4', 'c5', 'c6', 'cn']  # Names corresponding to each dataset
datas=[ext_work_of_ens_avg_greedy_c2,ext_work_of_ens_avg_greedy_c4,ext_work_of_ens_avg_greedy_c5,ext_work_of_ens_avg_greedy_c6,ext_work_of_ens_avg_greedy_cn]


for data,conn in zip(datas, dataset_names):
    num_time_steps = len(data)-2  # The number of rows in the data corresponds to the number of time steps
    t = np.arange(num_time_steps)  # Create a list of time steps [0, 1, 2, ..., num_time_steps-1]
    
    data=np.array(data[1:499])
    data_transposed = data.T  # Now rows are [x1_all_times, x2_all_times, ..., xN_all_times]
    
    plt.figure(figsize=(12, 8))
    #change this when analyzing pure dataset
    #plt.axhline(y=ytherm10, color='r', linestyle='--', label=f'y = {ytherm10}')

    # Plot each number as a function of t with specified colors
    for i, y_values in enumerate(data_transposed):
        color = colors[i % len(colors)]  # Cycle through the color list
        plt.plot(t, y_values, label=f'$Q_{i+1}$', color=color)
    
    # Add labels and legend
    plt.xlabel('Time (t)',fontsize=14)
    plt.ylabel('Value',fontsize=14)
    plt.title('$\Delta W(<\mathcal{Q}_i >)$ as a function of time',fontsize=16)
    plt.legend(fontsize=12)
    
    # Show the plot
    plt.savefig(f'Extractable work of ensemble averaged network {num}q_conn_{conn}_rule_R2_{dataset_label}.pdf', format='pdf', bbox_inches='tight')

    plt.show()

In [ ]:
#mimic
# Define a list of colors (you can choose any color names or hex codes)
#for 8Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2']
#for 10 Q
colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF']
#for 12 Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C']
#for 14Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C', '#C4BEB5', '#8E7BBA']
dataset_names = ['c2', 'c4', 'c5', 'c6', 'cn']  # Names corresponding to each dataset
datas=[ext_work_of_ens_avg_mimic_c2,ext_work_of_ens_avg_mimic_c4,ext_work_of_ens_avg_mimic_c5,ext_work_of_ens_avg_mimic_c6,ext_work_of_ens_avg_mimic_cn]


for data,conn in zip(datas, dataset_names):
    num_time_steps = len(data)-2  # The number of rows in the data corresponds to the number of time steps
    t = np.arange(num_time_steps)  # Create a list of time steps [0, 1, 2, ..., num_time_steps-1]
    
    data=np.array(data[1:499])
    data_transposed = data.T  # Now rows are [x1_all_times, x2_all_times, ..., xN_all_times]
    
    plt.figure(figsize=(12, 8))
    #change this when analyzing pure dataset
    #plt.axhline(y=ytherm10, color='r', linestyle='--', label=f'y = {ytherm10}')

    # Plot each number as a function of t with specified colors
    for i, y_values in enumerate(data_transposed):
        color = colors[i % len(colors)]  # Cycle through the color list
        plt.plot(t, y_values, label=f'$Q_{i+1}$', color=color)
    
    # Add labels and legend
    plt.xlabel('Time (t)',fontsize=14)
    plt.ylabel('Value',fontsize=14)
    plt.title('$\Delta W(<\mathcal{Q}_i >)$ as a function of time',fontsize=16)
    plt.legend(fontsize=12)
    
    # Show the plot
    plt.savefig(f'Extractable work of ensemble averaged network {num}q_conn_{conn}_rule_R3_{dataset_label}.pdf', format='pdf', bbox_inches='tight')

    plt.show()

In [ ]:
#random
# Define a list of colors (you can choose any color names or hex codes)
#for 8Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2']
#for 10 Q
colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF']
#for 12 Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C']
#for 14Q
#colors = ['#5B8FD8', '#FF6F61', '#6DBE45', '#D79F9D', '#7E7B9A','#F0E68C', '#008080', '#C2A2D2', '#D04E00', '#A8E6CF','#FFDAB9', '#B2B09C', '#C4BEB5', '#8E7BBA']
dataset_names = ['c2', 'c4', 'c5', 'c6', 'cn']  # Names corresponding to each dataset
datas=[ext_work_of_ens_avg_landscape_maximizes_c2,ext_work_of_ens_avg_landscape_maximizes_c4,ext_work_of_ens_avg_landscape_maximizes_c5,ext_work_of_ens_avg_landscape_maximizes_c6,ext_work_of_ens_avg_landscape_maximizes_cn]


for data,conn in zip(datas, dataset_names):
    num_time_steps = len(data)-2  # The number of rows in the data corresponds to the number of time steps
    t = np.arange(num_time_steps)  # Create a list of time steps [0, 1, 2, ..., num_time_steps-1]
    
    data=np.array(data[1:499])
    data_transposed = data.T  # Now rows are [x1_all_times, x2_all_times, ..., xN_all_times]
    
    plt.figure(figsize=(12, 8))
    #change this when analyzing pure dataset
    #plt.axhline(y=ytherm10, color='r', linestyle='--', label=f'y = {ytherm10}')

    # Plot each number as a function of t with specified colors
    for i, y_values in enumerate(data_transposed):
        color = colors[i % len(colors)]  # Cycle through the color list
        plt.plot(t, y_values, label=f'$Q_{i+1}$', color=color)
    
    # Add labels and legend
    plt.xlabel('Time (t)',fontsize=14)
    plt.ylabel('Value',fontsize=14)
    plt.title('$\Delta W(<\mathcal{Q}_i >)$ as a function of time',fontsize=16)
    plt.legend(fontsize=12)
    
    # Show the plot
    plt.savefig(f'Extractable work of ensemble averaged network {num}q_conn_{conn}_rule_R4_{dataset_label}.pdf', format='pdf', bbox_inches='tight')

    plt.show()